In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [2]:
# import treated data

xlsx = pd.ExcelFile('Feature_extracts well1.xlsx')
df = pd.read_excel(xlsx, 'PCAs')
dataset_pca= df.copy()



dataset_pca.pop('index')

# dataset_pca1= dataset_pca('PC1','PC1','PC3')
dataset_pca

,PC1,PC2,PC3
0,-3.398520,2.499091,0.178797
1,-3.403833,2.489971,0.105971
2,-3.451285,2.485884,0.033656
3,-3.515515,2.479439,-0.027975
4,-3.560541,2.492715,-0.047035
...,...,...,...
13852,0.962859,-2.179569,-2.287310
13853,0.938350,-2.179777,-2.362097
13854,0.894981,-2.185274,-2.387264
13855,0.858602,-2.230196,-2.407852


In [3]:
# import treated data

# xlsx_3 = pd.ExcelFile('Feature_extracts.xlsx')
df_2 = pd.read_excel(xlsx, 'target')
pca_target= df_2.copy()

pca_target

,RHOB,RHOB_r
0,-1.520406,2.24710
1,-1.418822,2.25870
2,-1.417071,2.25890
3,-1.454727,2.25460
4,-1.481874,2.25150
...,...,...
13852,0.924172,2.52625
13853,0.921107,2.52590
13854,0.918042,2.52555
13855,0.914977,2.52520


In [4]:

from sklearn import metrics, model_selection

train_input, test_input, train_target, test_target = model_selection.train_test_split(dataset_pca, pca_target['RHOB_r'], train_size=0.7)

print('train samples:', len(train_input))
print('test samples', len(test_input))

train samples: 9699
test samples 4158


In [17]:
def build_model(hp):
    model = keras.Sequential()

    
    for i in range(hp.Int('num_layers', 1, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=2,
                                            max_value=512,
                                            step=8,            
#                                             sampling="log"
                                           ),
                                            activation= 'relu',
                                            
#                                activation=hp.Choice('act_' + str(i), ['relu'])#, 'ELU', 'maxout', 'Leaky Relu' # adding other methods worsens the results
                               
#                                 kernel_regularizer=hp.Choice('act_' + str(i), ['l1', 'l2']) #is not helping at all
                               
                              ))
#         for i in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] 
#         model.add(layers.Dropout(rate=hp.Float('D_rate_'+ str(i),
#                                               min_value=0.0,
#                                               max_value=1.0,
#                                               step=0.1
# #                                               sampling="log"
#                                               ),       
#                                 ))

#         model.add(layers.Dropout(rate=hp.Choice('dropout rate_'+ str(i),
#                                              [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
#                                 ))
    model.add(layers.Dense(1,
#                            activation='linear'
                          ))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [0.15, 0.1, 1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mse'])
    return model

In [20]:
tuner = RandomSearch(
    build_model,
    objective=  'val_loss',   #   'val_mean_absolute_error', #(['loss', 'mse', 'val_loss', 'val_mse']
    max_trials=10,
    executions_per_trial=10,
    directory='Test_weights',
    project_name='ANNTuning')

INFO:tensorflow:Reloading Oracle from existing project Test_weights\ANNTuning\oracle.json
INFO:tensorflow:Reloading Tuner from Test_weights\ANNTuning\tuner0.json


In [21]:
tuner.search_space_summary()

Search space summary
Default search space size: 38
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 512, 'step': 8, 'sampling': None}
D_rate_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.15, 'conditions': [], 'values': [0.15, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 512, 'step': 8, 'sampling': None}
D_rate_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': 0.1, 'sampling': None}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 512, 'step': 8, 'sampling': None}
D_rate_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': 0.1, 'sampling': None}
units_3 (Int)
{'de

In [22]:
from tensorflow.keras.layers.experimental import preprocessing
#     #normalization
# normalizer=preprocessing.Normalization()
#     #then adapt it to the data
# norm_train_input= normalizer.adapt(np.array(train_input))
    
tuner.search(train_input, train_target,
             epochs=50,
             validation_data=(test_input, test_target))

Trial 9 Complete [00h 12m 15s]
val_loss: 0.030086887814104558

Best val_loss So Far: 0.029775709100067615
Total elapsed time: 01h 36m 49s
INFO:tensorflow:Oracle triggered exit


In [23]:
tuner.results_summary()

Results summary
Results in Test_weights\ANNTuning
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 11
units_0: 394
D_rate_0: 0.9
learning_rate: 0.001
units_1: 258
D_rate_1: 0.8
units_2: 322
D_rate_2: 0.30000000000000004
units_3: 250
D_rate_3: 0.2
units_4: 10
D_rate_4: 0.9
units_5: 442
D_rate_5: 0.7000000000000001
units_6: 202
D_rate_6: 0.5
units_7: 154
D_rate_7: 0.1
units_8: 82
D_rate_8: 0.5
units_9: 202
D_rate_9: 0.9
units_10: 354
D_rate_10: 0.6000000000000001
units_11: 130
D_rate_11: 0.1
units_12: 450
D_rate_12: 0.1
units_13: 410
D_rate_13: 0.2
units_14: 42
D_rate_14: 0.2
units_15: 106
D_rate_15: 0.9
units_16: 114
D_rate_16: 0.5
units_17: 234
D_rate_17: 0.4
units_18: 330
units_19: 306
Score: 0.029775709100067615
Trial summary
Hyperparameters:
num_layers: 9
units_0: 98
D_rate_0: 0.30000000000000004
learning_rate: 0.0001
units_1: 26
D_rate_1: 0.2
units_2: 114
D_rate_2: 0.30000000000000004
units_3: 418
D_rate_3: 1.0
units_4: 3

In [ ]:
models = tuner.get_best_models(num_models=2)
models

# Test run

In [20]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

# A function to hold preprocessor, layers, model and compiler 
def FFBackProp(inputs, output):
   

#     #normalization
#     normalizer=preprocessing.Normalization()
#     #then adapt it to the data
#     normalizer.adapt(np.array(inputs))
    
    
    model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32,activation="relu", input_shape=(2,), 
#                           kernel_regularizer="l2"
                         ),
    tf.keras.layers.Dropout(0.2),
        
    tf.keras.layers.Dense(64,activation="relu", 
#                           kernel_regularizer="l2"
                         ),
    tf.keras.layers.Dropout(0.2),
        
    tf.keras.layers.Dense(32,activation="relu", 
#                           kernel_regularizer="l2"
                         ),
    tf.keras.layers.Dropout(0.2),
        
#     tf.keras.layers.Dense(32,activation="relu", 
# #                           kernel_regularizer="l2"
#                          ),
#     tf.keras.layers.Dropout(0.2),
        
#     tf.keras.layers.Dense(64,activation="relu",
# #                           kernel_regularizer="l2"
#                          ),
#     tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(64,activation="relu",
# #                           kernel_regularizer="l2"
#                          ),
#     tf.keras.layers.Dropout(0.2),
        
    
        #output layer
    tf.keras.layers.Dense(1),
  ]
    )
    
    
#     compile the model
    model1.compile(optimizer='adam',
                loss='mean_absolute_error', #(from_logits=True),
                metrics=["mse"]
               )
    
    model1.summary()
    
    return model1
    
myFFBP= FFBackProp(train_input,train_target)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 32)                96        
_________________________________________________________________
dropout_16 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_18 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                

In [21]:
%%time
tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

myFFBPtraining = myFFBP.fit(
                            train_input, train_target,
                            # Calculate validation results on 30% of the training data
                            validation_split=0.3,


                            validation_data=(test_input, test_target),


                            # suppress logging
                            verbose=0,

#                             epochs=50,

                            batch_size= 40,


                            #early stopping
                            #the fuction to stop the trianing by tracking the validation loss
                            callbacks= keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)

                            #to store callbacks in logs
                        #     callbacks =[tb_callback]
)


myFFBP.evaluate(test_input, test_target,
#                 callbacks=[tb_callback],
                verbose=0, )

SyntaxError: invalid syntax (<ipython-input-21-b3e0381a5ee5>, line 20)

# plot results

In [ ]:
from matplotlib import pyplot as plt

def plot_loss(arg):
  plt.plot(arg.history['loss'], label='loss')
  plt.plot(arg.history['val_loss'], label='val_loss')
#   plt.ylim([0, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error [RHOB]')
  plt.legend()
  plt.grid(True)


plot_loss(myFFBPtraining)

In [ ]:
predicted = myFFBP.predict(test_input)

fig, ax = plt.subplots()
ax.scatter(test_target, predicted, alpha=0.5)

ax.set_xlabel('True Values')
ax.set_ylabel('Predicted')


In [ ]:
# using sklearn
from sklearn import metrics
print("r2 score: {}".format(metrics.r2_score(test_target_actual, test_target_predicted)))
print("mse: {}".format(metrics.mean_squared_error(test_target_actual, test_target_predicted)))
print("rmse: {}".format(np.sqrt(metrics.mean_squared_error(test_target_actual, test_target_predicted))))
print("mae: {}".format(metrics.mean_absolute_error(test_target_actual, test_target_predicted)))